# Estudo para a criação da baldeação da API

## Criação de um grafo fake

O primeiro passo, foi a criação de um grafo simulando paradas, para poder desenvolver a lógica do algoritmo.

In [6]:
import networkx as nx
import folium
import random
import pickle

# Criando um grafo com as coordenadas de exemplo em Brasília
G = nx.Graph()

# Adicionando nós (paradas de ônibus)
# Estas são coordenadas aleatórias próximas ao centro de Brasília
# Você pode substituir essas coordenadas pelas coordenadas reais das paradas de ônibus
bus_stops = [
    {'id': 1, 'lat': -15.793403, 'lon': -47.882778},
    {'id': 2, 'lat': -15.794768, 'lon': -47.881055},
    {'id': 3, 'lat': -15.796180, 'lon': -47.879300},
    {'id': 4, 'lat': -15.797451, 'lon': -47.877541},
]

for stop in bus_stops:
    G.add_node(stop['id'], lat=stop['lat'], lon=stop['lon'])

# Adicionando arestas (caminhos entre as paradas)
# Adicionando arestas aleatórias apenas para fins ilustrativos
edges = [(1, 2), (2, 3), (3, 4), (4, 1), (1, 3), (2, 4)]

for edge in edges:
    G.add_edge(*edge)

# Salvando o grafo
with open('bus_stops_graph.pkl', 'wb') as f:
    pickle.dump(G, f)

# Carregando o grafo
with open('bus_stops_graph.pkl', 'rb') as f:
    G_loaded = pickle.load(f)

# Criando um mapa usando folium
m = folium.Map(location=[-15.795574, -47.880671], zoom_start=15)

# Adicionando pontos de ônibus no mapa
for node, attributes in G_loaded.nodes(data=True):
    folium.Marker(
        location=[attributes['lat'], attributes['lon']],
        popup=f"Parada {node}",
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)

# Mostrar mapa
m.save("map.html")

## Parada mais próxima

O passo seguinte, foi calcular qual era a parada mais próxima, essa atividade não será feita na versão final, dado que o algoritmo receberá a parada que o usuário deverá pegar, não sendo necessário o cálculo da parada mais próxima, estamos fazendo aqui, para poder continuar o estudo do algoritmo.

Na primeira versão desse notebook, estavamos usando o grafo fake acima, agora, já usamos a versão final.

In [6]:
import networkx as nx
import folium
import pickle
import math

# Carregando o grafo salvo
with open('grafoDirecional.grafo', 'rb') as f:
    G_loaded = pickle.load(f)

# Função para calcular a distância entre duas coordenadas (latitude e longitude)
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371000  # raio da Terra em metros
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c
    return meters

# Função para encontrar a parada mais próxima e criar um mapa
def find_nearest_stop_and_create_map(coord):
    nearest_stop = None
    nearest_distance = float('inf')
  
    for node, attributes in G_loaded.nodes(data=True):
        stop_coord = (attributes['coords'].y, attributes['coords'].x)
        distance = haversine(coord, stop_coord)
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_stop = node
  
    m = folium.Map(location=[coord[0], coord[1]], zoom_start=15)
  
    # Adicionando a posição atual
    folium.Marker(
        location=coord,
        popup="Você está aqui",
        icon=folium.Icon(icon="star"),
    ).add_to(m)
  
    # Adicionando a parada mais próxima
    attributes = G_loaded.nodes[nearest_stop]
    folium.Marker(
        location=[attributes['coords'].y, attributes['coords'].x],
        popup=f"Parada mais próxima: {nearest_stop}",
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)
  
    # Adicionando linha entre a posição atual e a parada mais próxima
    folium.PolyLine([coord, [attributes['coords'].y, attributes['coords'].x]], color="blue", weight=2.5).add_to(m)
  
    m.save("nosso_nearest_stop_map.html")

# Teste da função com uma coordenada aleatória em Brasília
find_nearest_stop_and_create_map((-15.787128, -47.890883))

## Caminho a pé
Nesse estudo, procurei saber como o OSMNX poderia fazer o cálculo do percursso a pé, para fazer a baldeação.

In [7]:
import osmnx as ox
import networkx as nx
import folium

def calculate_route(start_point, end_point, graph):
    nearest_start = ox.distance.nearest_nodes(graph, X=[start_point[1]], Y=[start_point[0]])[0]
    nearest_end = ox.distance.nearest_nodes(graph, X=[end_point[1]], Y=[end_point[0]])[0]
    
    route = nx.shortest_path(graph, nearest_start, nearest_end, weight='length', method='dijkstra')
    
    m = folium.Map(location=[(start_point[0] + end_point[0]) / 2, (start_point[1] + end_point[1]) / 2], zoom_start=15)
    
    route_points = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in route]
    
    folium.PolyLine(route_points, color='blue').add_to(m)
    folium.Marker(location=start_point, popup='Start', icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=end_point, popup='End', icon=folium.Icon(color='red')).add_to(m)
    
    m.save('nosso_route_to_nearest_bus_stop.html')

# Baixar o mapa da área de interesse
# Neste exemplo, eu estou usando uma localização aproximada em Brasília
location_point = (-15.7980, -47.8929)
G = ox.graph_from_point(location_point, dist=2000, network_type='walk')

# Coordenadas de exemplo
start_point = (-15.7980, -47.8929)
end_point = (-15.793403, -47.882778)  # Coordenadas de uma parada de ônibus

# Calcular e mostrar a rota
calculate_route(start_point, end_point, G)


## Juntando as funções

Agora que estudamos como achar a parada mais próxima e calcular a baldeação a pé, iremos juntar as duas funções.

In [10]:
import osmnx as ox
import networkx as nx
import folium
import pickle
import math

# Carregando o grafo das paradas de ônibus
with open('grafoDirecional.grafo', 'rb') as f:
    bus_stops_graph = pickle.load(f)

# Função para calcular a distância haversine
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371000  # raio da Terra em metros
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c
    return meters

# Função para encontrar a parada de ônibus mais próxima
def find_nearest_stop(coord, graph):
    nearest_stop = None
    nearest_distance = float('inf')

    for node, attributes in graph.nodes(data=True):
        stop_coord = (attributes['coords'].y, attributes['coords'].x)
        distance = haversine(coord, stop_coord)
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_stop = node
    
    return graph.nodes[nearest_stop]['coords'].y, graph.nodes[nearest_stop]['coords'].x

# Função para calcular e desenhar a rota de caminhada
def calculate_route(start_point, end_point, graph):
    nearest_start = ox.distance.nearest_nodes(graph, X=[start_point[1]], Y=[start_point[0]])[0]
    nearest_end = ox.distance.nearest_nodes(graph, X=[end_point[1]], Y=[end_point[0]])[0]
    
    route = nx.shortest_path(graph, nearest_start, nearest_end, weight='length', method='dijkstra')
    
    m = folium.Map(location=[(start_point[0] + end_point[0]) / 2, (start_point[1] + end_point[1]) / 2], zoom_start=15)
    
    route_points = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in route]
    
    folium.PolyLine(route_points, color='blue').add_to(m)
    folium.Marker(location=start_point, popup='Start', icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=end_point, popup='End', icon=folium.Icon(color='red')).add_to(m)
    
    m.save('nosso_route_to_nearest_bus_stop.html')

# Coordenadas do ponto de partida (poderia ser a localização do usuário)
start_point = (-15.787128, -47.890883)

# Encontrar a parada de ônibus mais próxima usando o grafo das paradas de ônibus
end_point = find_nearest_stop(start_point, bus_stops_graph)

# Baixar o mapa da área de interesse para cálculo da rota de caminhada
G = ox.graph_from_point(start_point, dist=2000, network_type='walk')

# Calcular e mostrar a rota
calculate_route(start_point, end_point, G)

## Pontos

Agora, precisamos de todos os pontos que o OSMNX calcula para podermos retornar ao nosso cliente e ele fazer o caminho, dado que somos um motor, e não o frontend em si.

In [11]:
import osmnx as ox
import networkx as nx
import folium
import pickle
import math

# Carregando o grafo das paradas de ônibus
with open('grafoDirecional.grafo', 'rb') as f:
    bus_stops_graph = pickle.load(f)

# Função para calcular a distância haversine
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371000  # raio da Terra em metros
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c
    return meters

# Função para encontrar a parada de ônibus mais próxima
def find_nearest_stop(coord, graph):
    nearest_stop = None
    nearest_distance = float('inf')

    for node, attributes in graph.nodes(data=True):
        stop_coord = (attributes['coords'].y, attributes['coords'].x)
        distance = haversine(coord, stop_coord)
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_stop = node
    
    return graph.nodes[nearest_stop]['coords'].y, graph.nodes[nearest_stop]['coords'].x

# Função para calcular e desenhar a rota de caminhada
def calculate_route(start_point, end_point, graph):
    nearest_start = ox.distance.nearest_nodes(graph, X=[start_point[1]], Y=[start_point[0]])[0]
    nearest_end = ox.distance.nearest_nodes(graph, X=[end_point[1]], Y=[end_point[0]])[0]
    
    route = nx.shortest_path(graph, nearest_start, nearest_end, weight='length', method='dijkstra')
    
    m = folium.Map(location=[(start_point[0] + end_point[0]) / 2, (start_point[1] + end_point[1]) / 2], zoom_start=15)
    
    route_points = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in route]
    
    folium.PolyLine(route_points, color='blue').add_to(m)
    folium.Marker(location=start_point, popup='Start', icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=end_point, popup='End', icon=folium.Icon(color='red')).add_to(m)
    
    return m, route_points

# Coordenadas do ponto de partida (poderia ser a localização do usuário)
start_point = (-15.787128, -47.890883)

# Encontrar a parada de ônibus mais próxima usando o grafo das paradas de ônibus
end_point = find_nearest_stop(start_point, bus_stops_graph)

# Baixar o mapa da área de interesse para cálculo da rota de caminhada
G = ox.graph_from_point(start_point, dist=2000, network_type='walk')

# Calcular a rota
map_obj, route_coords = calculate_route(start_point, end_point, G)

# Salvar o mapa
map_obj.save('route_to_nearest_bus_stop_next.html')

# Agora você pode usar route_coords para qualquer propósito adicional
print(route_coords)


[(-15.7868496, -47.891253), (-15.7868335, -47.8913972), (-15.7868154, -47.8915434), (-15.7874336, -47.8915152), (-15.7874929, -47.8914347), (-15.7877226, -47.8914642), (-15.7877665, -47.89166), (-15.7882472, -47.8918116), (-15.7888044, -47.8919568), (-15.7892356, -47.8921281), (-15.7893192, -47.8919013), (-15.7896389, -47.8910394), (-15.7898633, -47.8904325)]


## Teste

Aqui, validei se o retorno dos pontos realmente formavam o mesmo desenho que o OSMNX puro fazia.

In [12]:
# Função para plotar um mapa usando apenas as coordenadas da rota
def plot_route_coords(route_coords):
    # Calcular o ponto médio das coordenadas da rota para centralizar o mapa
    avg_latitude = sum([coord[0] for coord in route_coords]) / len(route_coords)
    avg_longitude = sum([coord[1] for coord in route_coords]) / len(route_coords)

    m = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=15)
    
    folium.PolyLine(route_coords, color='blue').add_to(m)
    
    # Marcando o ponto de partida e chegada
    folium.Marker(location=route_coords[0], popup='Start', icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=route_coords[-1], popup='End', icon=folium.Icon(color='red')).add_to(m)

    return m

# Usando a função para plotar o mapa com as coordenadas da rota
route_map = plot_route_coords(route_coords)
route_map.save('nosso_route_coords_map.html')